# 第6章 Gated RNN

来了LSTM

![](./img/bq8.jpg)

## RNN的问题

### 梯度消失和梯度爆炸

示例

![](./img/6-3.png)

RNN梯度传播过程

![](./img/6-4.png)

梯度变小（梯度消失）或者梯度变大（梯度爆炸）

### 梯度消失和梯度爆炸的原因

第T个正确接标签是Tom

T = 19

关注RNN在时间方向上的传播

![](./img/6-5.png)

- tanh

- +

- Mat Mul

其中加法不用管，导数不动

重点看tanh和Mat Mul

#### tanh

![](./img/6-6.png)

虚线是 y = tanh(x) 的导数

从图中可以看出，它的值小于1.0，并且随着 x 远离 0，它的值在变小。

这意味着，当反向传播的梯度经过tanh 节点时，它的值会越来越小。

因此，如果经过 tanh 函数 T 次，则梯度也会减小 T 次。

使用ReLU可以改善性能

#### MatMul

![](./img/6-7.png)

In [ ]:
# ch06/rnn_gradient_graph.py
import numpy as np
import matplotlib.pyplot as plt


N = 2  # mini-batch的大小
H = 3  # 隐藏状态向量的维数
T = 20  # 时序数据的长度

dh = np.ones((N, H)) # 初始化dh 所有元素都为1的矩阵

np.random.seed(3)

Wh = np.random.randn(H, H)

norm_list = []
for t in range(T):
    dh = np.dot(dh, Wh.T)
    # 将每一步导数的大小 L2范数 加入到norm_list中
    norm = np.sqrt(np.sum(dh**2)) / N  # 对所有元素的平方求平方根
    
    norm_list.append(norm)

print(norm_list)

In [ ]:
Wh = np.random.randn(H, H)
Wh

画出图像

In [ ]:
# 绘制图形
plt.plot(np.arange(len(norm_list)), norm_list)
plt.xticks([0, 4, 9, 14, 19], [1, 5, 10, 15, 20])
plt.xlabel('time step')
plt.ylabel('norm')
plt.show()

可以看到梯度大小随时间步长呈指数级增加，这就是梯度爆炸

exploding gradients

如果发生梯度爆炸，最终会导致溢出，出现Nan，not a number之类的值

神经网络的学习将无法正确进行

下面改变初始值

In [ ]:
Wh = np.random.randn(H, H) * 0.5
Wh

In [ ]:
# ch06/rnn_gradient_graph.py
import numpy as np
import matplotlib.pyplot as plt


N = 2  # mini-batch的大小
H = 3  # 隐藏状态向量的维数
T = 20  # 时序数据的长度 

dh = np.ones((N, H)) # 初始化dh 所有元素都为1的矩阵

np.random.seed(3)

Wh = np.random.randn(H, H) * 0.5

norm_list = []
for t in range(T):
    dh = np.dot(dh, Wh.T)
    # 将每一步导数的大小 L2范数 加入到norm_list中
    norm = np.sqrt(np.sum(dh**2)) / N  # 对所有元素的平方求平方根
    
    norm_list.append(norm)

print(norm_list)

In [ ]:
# 绘制图形
plt.plot(np.arange(len(norm_list)), norm_list)
plt.xticks([0, 4, 9, 14, 19], [1, 5, 10, 15, 20])
plt.xlabel('time step')
plt.ylabel('norm')
plt.show()

这次梯度呈指数级减小，这就是梯度消失（vanishing gradients）

如果发生梯度消失，梯度将迅速变小。

一旦梯度变小，权重梯度不能被更新，模型就会无法学习长期的依赖关系。

本质是因为

$$W_h^n$$

如果 Wh 是标量：

- 当 Wh 大于 1 时，梯度呈指数级增加；

- 当 Wh小于 1 时，梯度呈指数级减小。

如果 Wh 是矩阵呢

则使用矩阵的奇异值，矩阵的奇异值表示数据的离散程度。

根据这个奇异值（更准确地说是多个奇异值中的最大值）是否大于 1，可以预测梯度大小的变化。

### 梯度爆炸的对策

梯度剪裁 gradients clipping

![](./img/6-9.png)

gˆ ：所有参数的梯度L2范数整合成一个数

    比如，当某个模型有 W1和 W2两个参数时，gˆ 就是这两个参数对应的梯度 dW1和 dW2的组合。

threshold：阈值

如果梯度的 L2 范数 gˆ 大于或等于阈值，就按上述方法修正梯度，这就是梯度裁剪。

如你所见，虽然这个方法很简单，但是在许多情况下效果都不错。



In [ ]:
# coding: utf-8
import numpy as np

dW1 = np.random.rand(3, 3) * 10
dW1

In [ ]:
dW2 = np.random.rand(3, 3) * 10
dW2

In [ ]:
grads = [dW1, dW2]
grads

In [ ]:
max_norm = 5.0

In [ ]:
total_norm = 0

In [ ]:
grad0 = grads[0]
grad0

In [ ]:
grad0 ** 2

In [ ]:
total_norm += np.sum(grad0 ** 2)
total_norm

In [ ]:
grad1 = grads[1]

In [ ]:
total_norm += np.sum(grad0 ** 2)
total_norm

In [ ]:
total_norm = np.sqrt(total_norm)
total_norm

In [ ]:
total_norm + 1e-6

In [ ]:
rate = max_norm / (total_norm + 1e-6)
rate

In [ ]:
if rate < 1:
    for grad in grads:
        grad *= rate
grads

In [ ]:
0.2539 * 6.49144048

In [ ]:
# coding: utf-8
import numpy as np


dW1 = np.random.rand(3, 3) * 10
dW2 = np.random.rand(3, 3) * 10
grads = [dW1, dW2]
max_norm = 5.0


def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    print('rate:', rate)
    if rate < 1:
        for grad in grads:
            grad *= rate

x1 = dW1.flatten()
print('before:', dW1.flatten())
clip_grads(grads, max_norm)
x2 = dW1.flatten()
print('after:', dW1.flatten())

In [ ]:
plt.plot(x1)
plt.plot(x2)
plt.xlabel('time step')
plt.show()

## 梯度消失和LSTM

Gated RNN，其中最具代表性的LSTM和GRU

本节关注LSTM

介绍顺序由外而内，由大到小

### LSTM的接口

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    
**tanh与sigmoid**

tanh的输出是−1.0 ~ 1.0的实数。我们可以认为这个−1.0 ~ 1.0的数值表示某种被编码的“信息”的强弱（程度）。

而sigmoid 函数的输出是0.0~1.0的实数，表示数据流出的比例。

因此，在大多数情况下，门使用sigmoid函数作为激活函数，而包含实质信息的数据则使用tanh函数作为激活函数。

### 新的记忆单元

![](./img/6-18.png)

### LSTM的梯度流动

上面我们介绍了 LSTM 的结构，那么，为什么它不会引起梯度消失呢？

其原因可以通过观察记忆单元 c 的反向传播来了解（图 6-19）

![](./img/6-19.png)

在图 6-19 中，我们仅关注记忆单元，绘制了它的反向传播。

此时，记忆单元的反向传播仅流过“+”和“×”节点。

“+”节点将上游传来的梯度原样流出，所以梯度没有变化（退化）。

而“×”节点的计算并不是矩阵乘积，而是对应元素的乘积（阿达玛积）。

顺便说一下，在之前的 RNN 的反向传播中，我们使用相同的权重矩阵重复了多次矩阵乘积计算，由此导致了梯度消失（或梯度爆炸）。

而这里的 LSTM 的反向传播进行的不是矩阵乘积计算，而是对应元素的乘积计算，而且每次都会基于不同的门值进行对应元素的乘积计算。

这就是它不会发生梯度消失（或梯度爆炸）的原因。

图 6-19 的“×”节点的计算由遗忘门控制（每次输出不同的门值）。

遗忘门认为“应该忘记”的记忆单元的元素，其梯度会变小；

而遗忘门认为“不能忘记”的元素，其梯度在向过去的方向流动时不会退化。

因此，可以期待记忆单元的梯度（应该长期记住的信息）能在不发生梯度消失的情况下传播。

从以上讨论可知，LSTM的记忆单元不会（难以）发生梯度消失。

因此，可以期待记忆单元能够保存（学习）长期的依赖关系。

<div class="alert alert-danger alertdanger" style="margin-top: 20px">

**LSTM**
    
是Long Short-Term Memory（长短期记忆）的缩写，意思是可以长（Long）时间维持短期记忆（Short-Term Memory）

## LSTM的实现

下面，我们来实现 LSTM。

这里将进行单步处理的类实现为 LSTM 类，将整体处理 T 步的类实现为 TimeLSTM 类。

现在我们先来整理一下 LSTM 中进行的计算，如下所示：

![](./img/6.6.png)

以上就是 LSTM 进行的计算。

这里需要注意式 (6.6) 中的 4 个仿射变换。

这里的仿射变换是指 xWx + hWh + b 这样的式子。

式 (6.6) 中通过 4 个式子分别进行仿射变换，但其实可以整合为通过 1 个式子进行，如图 6-20 所示。

![](./img/6-20.png)

在图 6-20 中，4 个权重（或偏置）被整合为了 1 个。

如此，原本单独执行 4 次的仿射变换通过 1 次计算即可完成，可以加快计算速度。

这是因为矩阵库计算“大矩阵”时通常会更快，而且通过将权重整合到一起管理，源代码也会更简洁。

假设 Wx、Wh 和 b 分别包含 4 个权重（或偏置），此时 LSTM 的计算图如图 6-21 所示。

![](./img/6-21.png)

先一起执行 4 个仿射变换。

然后，基于 slice 节点，取出 4 个结果。

这个 slice 节点很简单，它将仿射变换的结果（矩阵）均等地分成 4 份，然后取出内容。

在 slice 节点之后，数据流过激活函数（sigmoid函数或 tanh 函数），进行上一节介绍的计算

In [3]:
# common/time_layers.py

class LSTM:
    def __init__(self, Wx, Wh, b):
        '''

        Parameters
        ----------
        Wx: 输入`x`用的权重参数（整合了4个权重）
        Wh: 隐藏状态`h`用的权重参数（整合了4个权重）
        b: 偏置（整合了4个偏置）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        # 它的参数接收当前时刻的输入 x
        # 上一时刻的隐藏状态 h_prev，以及上一时刻的记忆单元 c_prev
        
        Wx, Wh, b = self.params
        N, H = h_prev.shape
        
        # 变量 Wx、Wh 和 b 保存的是 4个权重
        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

矩阵形状

![](./img/6-22.png)

总感觉这个图写错了呀

![](./img/6-23.png)

![](./img/6-24.png)

In [4]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 20, 35, 100
        H = Wh.shape[0] # 100

        self.layers = []
        hs = np.empty((N, T, H), dtype='f') # 20*35*100

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f') # # 20*100
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')# 20*100

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c) # 20*100 # 20*100 # 20*100
            hs[:, t, :] = self.h

            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0

        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        return dxs

    def set_state(self, h, c=None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None

![](./img/6-25.png)

## 使用LSTM的语言模型

跟上一章基本一样

不同的是上一张章用的Time RNN，这一章换成Time LSTM

![](./img/6-26.png)

In [5]:
import os
import pickle
import numpy as np

key_file = {
    'train':'ptb.train.txt',
    'test':'ptb.test.txt',
    'valid':'ptb.valid.txt'
}
save_file = {
    'train':'ptb.train.npy',
    'test':'ptb.test.npy',
    'valid':'ptb.valid.npy'
}
vocab_file = 'ptb.vocab.pkl'

dataset_dir = os.path.join(os.path.abspath('.'), 'dataset')
dataset_dir

'D:\\06Jupyter\\03_code\\18深度学习进阶自然语言处理\\神仔的代码\\dataset'

In [6]:
def load_vocab():
    vocab_path = dataset_dir + '/' + vocab_file

    if os.path.exists(vocab_path):
        with open(vocab_path, 'rb') as f:
            word_to_id, id_to_word = pickle.load(f)
        return word_to_id, id_to_word

    word_to_id = {}
    id_to_word = {}
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name


    words = open(file_path).read().replace('\n', '<eos>').strip().split()

    for i, word in enumerate(words):
        if word not in word_to_id:
            tmp_id = len(word_to_id)
            word_to_id[word] = tmp_id
            id_to_word[tmp_id] = word

    with open(vocab_path, 'wb') as f:
        pickle.dump((word_to_id, id_to_word), f)

    return word_to_id, id_to_word

In [7]:
def load_data(data_type='train'):
    '''
        :param data_type: 数据的种类：'train' or 'test' or 'valid (val)'
        :return:
    '''
    if data_type == 'val': data_type = 'valid'
    save_path = dataset_dir + '/' + save_file[data_type]

    word_to_id, id_to_word = load_vocab()

    if os.path.exists(save_path):
        corpus = np.load(save_path)
        return corpus, word_to_id, id_to_word

    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name

    words = open(file_path).read().replace('\n', '<eos>').strip().split()
    corpus = np.array([word_to_id[w] for w in words])

    np.save(save_path, corpus)
    return corpus, word_to_id, id_to_word

In [8]:
# common/base_model.py

# import sys
# sys.path.append('..')
import os
import pickle
from common.np import *
from common.util import to_gpu, to_cpu

class BaseModel:
    def __init__(self):
        self.params, self.grads = None, None

    def forward(self, *args):
        raise NotImplementedError

    def backward(self, *args):
        raise NotImplementedError

    def save_params(self, file_name=None):
        if file_name is None:
            file_name = self.__class__.__name__ + '.pkl'

        params = [p.astype(np.float16) for p in self.params]
        if GPU:
            params = [to_cpu(p) for p in params]

        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name=None):
        if file_name is None:
            file_name = self.__class__.__name__ + '.pkl'

        if '/' in file_name:
            file_name = file_name.replace('/', os.sep)

        if not os.path.exists(file_name):
            raise IOError('No file: ' + file_name)

        with open(file_name, 'rb') as f:
            params = pickle.load(f)

        params = [p.astype('f') for p in params]
        if GPU:
            params = [to_gpu(p) for p in params]

        for i, param in enumerate(self.params):
            param[...] = params[i]

该 类 实 现 了 save_params()和 load_params()方法。

因此，通过继承 BaseModel类，也能获得数据读写的功能。

另外，BaseModel类的实现还进行了优化，以支持 GPU 和进行缩位（使用 16 位浮点数存储）。

In [103]:
# ch06/rnnlm.py
# coding: utf-8

# import sys
# sys.path.append('..')
from common.time_layers import *
# from common.base_model import BaseModel


class Rnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=100, hidden_size=100):
        V, D, H = vocab_size, wordvec_size, hidden_size # 10000 100 100
        rn = np.random.randn

        # 初始化权重
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f') # 100 4*100
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f') # 100 4*400
        lstm_b = np.zeros(4 * H).astype('f') # 4*100
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # 生成层
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]

        # 将所有的权重和梯度整理到列表中
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        for layer in self.layers:
            print('输入：', xs.shape)
            print('{}'.format(layer))
            xs = layer.forward(xs)
            print('输出：', xs.shape)
        return xs

    def forward(self, xs, ts):
        score = self.predict(xs)
        print('输入score：', score.shape)
        print('loss_layer')
        loss = self.loss_layer.forward(score, ts)
        print('输出:', loss)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.lstm_layer.reset_state()



在 PTB 数据集上学习这个网络

In [104]:
# ch06/train_rnnlm.py

# coding: utf-8
# import sys
# sys.path.append('..')
from common.optimizer import SGD
# from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
# from dataset import ptb
# from rnnlm import Rnnlm

In [105]:
# 设定超参数
batch_size = 20
wordvec_size = 100
hidden_size = 100  # RNN的隐藏状态向量的元素个数
time_size = 35  # RNN的展开大小
lr = 20.0
max_epoch = 4
max_grad = 0.25

In [106]:
# 读入训练数据
corpus, word_to_id, id_to_word = load_data('train')

In [81]:
corpus

array([ 0,  1,  2, ..., 39, 26, 24], dtype=int64)

In [82]:
len(corpus)

929589

In [83]:
word_to_id

{'aer': 0,
 'banknote': 1,
 'berlitz': 2,
 'calloway': 3,
 'centrust': 4,
 'cluett': 5,
 'fromstein': 6,
 'gitano': 7,
 'guterman': 8,
 'hydro-quebec': 9,
 'ipo': 10,
 'kia': 11,
 'memotec': 12,
 'mlx': 13,
 'nahb': 14,
 'punts': 15,
 'rake': 16,
 'regatta': 17,
 'rubens': 18,
 'sim': 19,
 'snack-food': 20,
 'ssangyong': 21,
 'swapo': 22,
 'wachter': 23,
 '<eos>': 24,
 'pierre': 25,
 '<unk>': 26,
 'N': 27,
 'years': 28,
 'old': 29,
 'will': 30,
 'join': 31,
 'the': 32,
 'board': 33,
 'as': 34,
 'a': 35,
 'nonexecutive': 36,
 'director': 37,
 'nov.': 38,
 'mr.': 39,
 'is': 40,
 'chairman': 41,
 'of': 42,
 'n.v.': 43,
 'dutch': 44,
 'publishing': 45,
 'group': 46,
 'rudolph': 47,
 'and': 48,
 'former': 49,
 'consolidated': 50,
 'gold': 51,
 'fields': 52,
 'plc': 53,
 'was': 54,
 'named': 55,
 'this': 56,
 'british': 57,
 'industrial': 58,
 'conglomerate': 59,
 'form': 60,
 'asbestos': 61,
 'once': 62,
 'used': 63,
 'to': 64,
 'make': 65,
 'kent': 66,
 'cigarette': 67,
 'filters': 68,
 'h

In [84]:
id_to_word

{0: 'aer',
 1: 'banknote',
 2: 'berlitz',
 3: 'calloway',
 4: 'centrust',
 5: 'cluett',
 6: 'fromstein',
 7: 'gitano',
 8: 'guterman',
 9: 'hydro-quebec',
 10: 'ipo',
 11: 'kia',
 12: 'memotec',
 13: 'mlx',
 14: 'nahb',
 15: 'punts',
 16: 'rake',
 17: 'regatta',
 18: 'rubens',
 19: 'sim',
 20: 'snack-food',
 21: 'ssangyong',
 22: 'swapo',
 23: 'wachter',
 24: '<eos>',
 25: 'pierre',
 26: '<unk>',
 27: 'N',
 28: 'years',
 29: 'old',
 30: 'will',
 31: 'join',
 32: 'the',
 33: 'board',
 34: 'as',
 35: 'a',
 36: 'nonexecutive',
 37: 'director',
 38: 'nov.',
 39: 'mr.',
 40: 'is',
 41: 'chairman',
 42: 'of',
 43: 'n.v.',
 44: 'dutch',
 45: 'publishing',
 46: 'group',
 47: 'rudolph',
 48: 'and',
 49: 'former',
 50: 'consolidated',
 51: 'gold',
 52: 'fields',
 53: 'plc',
 54: 'was',
 55: 'named',
 56: 'this',
 57: 'british',
 58: 'industrial',
 59: 'conglomerate',
 60: 'form',
 61: 'asbestos',
 62: 'once',
 63: 'used',
 64: 'to',
 65: 'make',
 66: 'kent',
 67: 'cigarette',
 68: 'filters',
 69

In [85]:
corpus_test, _, _ = load_data('test')

In [86]:
corpus_test

array([142,  78,  54, ...,  87, 214,  24], dtype=int64)

In [87]:
len(corpus_test)

82430

In [88]:
vocab_size = len(word_to_id)
vocab_size

10000

In [89]:
xs = corpus[:-1]
ts = corpus[1:]

In [90]:
len(xs)

929588

In [109]:
# 生成模型
model = Rnnlm(vocab_size, wordvec_size, hidden_size) # 10000  100  100
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

In [107]:
import time
import numpy
import time
import matplotlib.pyplot as plt
from common.np import *  # import numpy as np
from common.util import clip_grads
class RnnlmTrainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        self.time_idx = None
        self.ppl_list = None
        self.eval_interval = None
        self.current_epoch = 0

    def get_batch(self, x, t, batch_size, time_size):
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')

        data_size = len(x)
        jump = data_size // batch_size
        offsets = [i * jump for i in range(batch_size)]  # mini-batch的各笔样本数据的开始位置

        for time in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, time] = x[(offset + self.time_idx) % data_size]
                batch_t[i, time] = t[(offset + self.time_idx) % data_size]
            self.time_idx += 1
        return batch_x, batch_t

    def fit(self, xs, ts, max_epoch=10, batch_size=20, time_size=35,
            max_grad=None, eval_interval=20):
        data_size = len(xs)
        max_iters = data_size // (batch_size * time_size)
        self.time_idx = 0
        self.ppl_list = []
        self.eval_interval = eval_interval
        model, optimizer = self.model, self.optimizer
        total_loss = 0
        loss_count = 0

        start_time = time.time()
        for epoch in range(max_epoch):
            for iters in range(max_iters):
                batch_x, batch_t = self.get_batch(xs, ts, batch_size, time_size)
                print(batch_x)
                print('------------------------------------')
                print(batch_t)
                print('------------------------------------')

                # 计算梯度，更新参数
                
                loss = model.forward(batch_x, batch_t)
                print(loss)
                print('########################################')
                model.backward()
                params, grads = remove_duplicate(model.params, model.grads)  # 将共享的权重整合为1个
                if max_grad is not None:
                    clip_grads(grads, max_grad)
                optimizer.update(params, grads)
                total_loss += loss
                loss_count += 1

                # 评价困惑度
                if (eval_interval is not None) and (iters % eval_interval) == 0:
                    ppl = np.exp(total_loss / loss_count)
                    elapsed_time = time.time() - start_time
                    print('| epoch %d |  iter %d / %d | time %d[s] | perplexity %.2f'
                          % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, ppl))
                    self.ppl_list.append(float(ppl))
                    total_loss, loss_count = 0, 0

            self.current_epoch += 1

    def plot(self, ylim=None):
        x = numpy.arange(len(self.ppl_list))
        if ylim is not None:
            plt.ylim(*ylim)
        plt.plot(x, self.ppl_list, label='train')
        plt.xlabel('iterations (x' + str(self.eval_interval) + ')')
        plt.ylabel('perplexity')
        plt.show()

In [108]:
def remove_duplicate(params, grads):
    '''
    将参数列表中重复的权重整合为1个，
    加上与该权重对应的梯度
    '''
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 在共享权重的情况下
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 加上梯度
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)
                # 在作为转置矩阵共享权重的情况下（weight tying）
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)

                if find_flg: break
            if find_flg: break

        if not find_flg: break

    return params, grads


应用梯度裁剪进行学习
处使用 RnnlmTrainer 类进行模
型的学习。RnnlmTrainer 类的 fit() 方法求模型的梯度，更新模型的参数。
另外，在方法内部，通过指定 max_grad 参数，从而应用梯度裁剪。顺便说一
下，fit() 方法内部进行的实现如下所示（这里给出的是伪代码）。
处的 fit() 方法的参数 eval_interval=20，每 20 次迭代对
困惑度进行 1 次评价。因为这次的数据量很大，所以没有对每个 epoch 进行
评价，而是每 20 次迭代评价 1 次。后面我们会将评价结果用 plot() 方法绘
制成图。

In [110]:
trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,
            eval_interval=20)
trainer.plot(ylim=(0, 500))

[[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
    14   15   16   17   18   19   20   21   22   23   24   25   26   27
    28   29   30   31   32   33   34]
 [  93  718  590 1569   35 4979   95   87  507   93   78  718   26 2966
   467   35 4979  119 2862   64 1177 2640  861 1449   26  956 5130   98
    24   32 2361   34   78   54  461]
 [  27  930   42  109  241  935  181  978   42   26 4255  119   27  930
  1652   24 2594  170   35 1624  470  189   69  414 5398  416   27  187
    24   26   26   54 3467  312   48]
 [  48 1523  180   24   93 2512 1883  623  220 1882 2745   64  979 4273
  3817  774  225   98   24 1116 1487 1757  718  220  368  520   26  169
  1967 3574  757 1487 4892   24   48]
 [ 160 1112  108 1321  373   42 1832 2943 5865  229 4381 4778   35   60
    42 1522  938 1036 4382 1972   24  315  337   26   34   32  362 7385
   152   48  410 5826 2480   64   32]
 [4434   32  431 7796  303 1871 8806  840  415  590 4004   26 1656   26
  3320   42 1911  

输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 10000)
输入score： (20, 35, 10000)
loss_layer
输出: 8.918362862723214
8.918362862723214
########################################
[[  37   42   56   57   58   59   24   35   60   42   61   62   63   64
    65   66   67   68   69   70   35   71   72   42   73   74   75   35
    46   42   76   77   64   78   79]
 [ 718  220  971   64  660   32 1515  531   32  462   40  124   64 2273
  4586   35 4979  590   93 2966  718  220   26  109 2862   64 1177  718
   220  709 5131   78  718  220 1621]
 [ 949   93   40  228  229 2305 4240  119 6785  231 1332  119  815  798
  1439 3305   24  353  687   27   27   24   32 1637  237   48 1313  496
   337  338  849  159   35 5231   64]
 [  32 4864   48  180  489  128 2487   93  225   98  718 1693   26 1003
    24  315  225   98   93   88  232  213 2233   32  189  646  718 4339
    35  502   42 5522   42 1879   42]
 [ 245   40 2680  119 1317   42  687 3912  128  390   42   32 1

输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 10000)
输入score： (20, 35, 10000)
loss_layer
输出: 10.456272321428571
10.456272321428571
########################################
[[  26   99   32  100   42  101  102   26  103   93  104   66  105  106
   107   26  108  109   26   67   68  108   27   24  110  111  112  113
    83   79   80   35  114   81   32]
 [ 451 5134   93  718  108 1168   64   32 1056 5135  840 1165   32  798
    64  660   32  462   42  128 4955  531   32  886  507 1216  113  124
    64 1260  108   35 1384  108   32]
 [  27   27   27 1982   27   27   27 1012 5216  852   27   27   27  744
    24 1506 1706   75 2490 1375  181 5232  805  108  178   42  416   27
   468  467   79   24 3013 5233 5234]
 [ 462   87  275 1624  507 1993  229 1375  167  229 6863  545   24 3660
   892   27 3662  930   48  718 3142  498 5655 6109   64 1832   79   80
   416   27  468  854   24  237 6109]
 [  26  209   24   32  403  119 4090 4068 2303 3248  133  128 3108   42
    35  350  467  133  41

输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 10000)
输入score： (20, 35, 10000)
loss_layer
输出: 10.252341657366072
10.252341657366072
########################################
[[ 133   28   81  134  135  136   42   61  137  138  139  140   24  141
    40  142   61  108  143  144  145   24  146   26  147   32   82  148
   149   32   76  113  150   42  138]
 [ 315 2640   26   98   93   32 1371  227  225 5137   35  625  337 5133
    54  124   64   65   32 2361   88   79 4188   24  152  353   32  523
   461   35   26 5133   93  718 1165]
 [2272   64 1769  152  935  936 5237   24 2490  180 1276 1097  229  536
  5238 4829  103   27   27   27   64   27  363   27   27   27   64   27
   363   27   27   27   64   27  363]
 [  26 7348   24   32  189  119   41   39   26   48   35   46   42 7450
  1386  160   32 3153   42   26 7788 7789  230 7790   35 1265 1106 1343
   108 3259 1896   24 3662  840   40]
 [ 710  351  119 8380   42 1637 6810 3194  596  394  408  405

输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 100)
输入： (20, 35, 100)
输出： (20, 35, 10000)
输入score： (20, 35, 10000)
loss_layer
输出: 8.003076171875
8.003076171875
########################################
[[  42   82  169   32  170   73  165   48   32  171  172   42  173  174
    48  164  174   24   32   26  175   98   61   54   63  108  176  177
   178  108  179  180  181   32   68]
 [ 845 1666   24   32 1371 1134   32   26 5133  229  109 5128 4135   54
  3120  160 3432 1068 3687  103   41 1552 5138  169 3482   64  885  301
   649 4955   24   26  347   24  316]
 [4207  108 5241   42  416   27   27   27   27  363   27   27   27  363
    27   27   27  363   24 4313   42 4255   27   27  247 1028   27   27
   652  790   27   27  195  790   27]
 [ 230 3644   24  495 3081   87 3662   98  109   33   40  251  124   64
   782  134   32  351  424  528   64 4131   64   32  852   24   32   33
    69   27 1088  363   64 4131   24]
 [2572  108  109  180  323  181  478 3331   24  373   32 1195  154 46

KeyboardInterrupt: 

运行时间比较长，我把在pycharm里面运行的结果截个图

![](./img/6-38.png)

In [ ]:
# 基于测试数据进行评价
model.reset_state() # 此时需要先重置模型的状态（LSTM 的隐藏状态和记忆单元）
ppl_test = eval_perplexity(model, corpus_test) # eval_perplexity() 在 common/util.py 中已经实现，直接使用即可

In [ ]:
print('test perplexity: ', ppl_test)

In [ ]:
# 保存参数
model.save_params()

处将学习好的参数保存到外部文件。在下一章生成句子时，将会使用这些学习好的权重参数

## 进一步改进RNNLM

### LSTM层的多层化

在使用 RNNLM 创建高精度模型时，加深 LSTM 层（叠加多个 LSTM层）的方法往往很有效。

之前我们只用了一个 LSTM 层，通过叠加多个层，可以提高语言模型的精度。

比如下面这个图RNNLM 使用了两个LSTM 层

![](./img/6-29.png)

在 PTB 数据集上学习语言模型的情况下，当LSTM 的层数为 2 ～ 4 时，可以获得比较好的结果

据报道，谷歌翻译中使用的 GNMT 模型 是叠加了 8 层 LSTM 的网络。

如该例所示，如果待解决的问题很难，又能准备大量的训练数据，
就可以通过加深 LSTM 层来提高精度。

### 基于Dropout抑制过拟合

加深层往往会发生过拟合（overfitting）。

抑制过拟合已有既定的方法：

一是增加训练数据；

二是降低模型的复杂度。我们会优先考虑这两个方法。

除此之外，对模型复杂度给予惩罚的正则化也很有效。

比如，L2 正则化会对过大的权重进行惩罚。

此外，像 Dropout，在训练时随机忽略层的一部分（比如50%）神经元，也可以被视为一种正则化。

本节我们将仔细研究Dropout，并将其应用于 RNN。

![](./img/6-30.png)

Dropout 随机选择一部分神经元，然后忽略它们，停止向前传递信号。

这种“随机忽视”是一种制约，可以提高神经网络的泛化能力。

我们在前作《深度学习入门：基于 Python 的理论与实现》中已经实现了 Dropout。

当时我们给出了在激活函数后插入Dropout 层的示例，并展示了它有助于抑制过拟合。

![](./img/6-31.png)

那么，在使用 RNN 的模型中，应该将 Dropout 层插入哪里呢？

首先可以想到的是插入在 LSTM 层的时序方向上，如图 6-32 所示。

不过答案是，这并不是一个好的插入方式。

![](./img/6-32.png)

如果在时序方向上插入 Dropout，那么当模型学习时，随着时间的推移，信息会渐渐丢失。

也就是说，因 Dropout 产生的噪声会随时间成比例地积累。

考虑到噪声的积累，最好不要在时间轴方向上插入 Dropout。

因此，如图 6-33 所示，我们在深度方向（垂直方向）上插入 Dropout 层。

![](./img/6-33.png)

这样一来，无论沿时间方向（水平方向）前进多少，信息都不会丢失。

Dropout 与时间轴独立，仅在深度方向（垂直方向）上起作用。

如前所述，“常规的 Dropout”不适合用在时间方向上。

但是，最近的研究提出了多种方法来实现时间方向上的 RNN 正则化。比如，文献 [36] 中
提出的“变分 Dropout”（variational dropout）就被成功地应用在了时间
方向上。

除了深度方向，变分 Dropout 也能用在时间方向上，从而进一步提高
语言模型的精度。

它的机制是同一层的 Dropout 使用相同的 mask。这里所说的 mask 是指决定是否传递数据的随机布尔值。

![](./img/6-34.png)

通过同一层的 Dropout 共用 mask，mask 被“固定”。

如此一来，信息的损失方式也被“固定”，所以可以避免常规 Dropout 发生的指数级信息损失。

### 权重共享

改进语言模型有一个非常简单的技巧，那就是权重共享（weight tying）。

weight tying 可以直译为“权重绑定”。


![](./img/6-35.png)

如图所示，绑定（共享）Embedding 层和 Affine 层的权重的技巧在于权重共享。

通过在这两个层之间共享权重，可以大大减少学习的参数数量。

尽管如此，它仍能提高精度。真可谓一石二鸟！

现在，我们来考虑一下权重共享的实现。

这里，假设词汇量为 V，LSTM 的隐藏状态的维数为 H，则 Embedding 层的权重形状为 V × H，Affine 层的权重形状为 H × V。

此时，如果要使用权重共享，只需将Embedding 层权重的转置设置为 Affine 层的权重。

这个非常简单的技巧可以带来出色的结果。

### 更好的RNNLM的实现

至此，我们介绍了 RNNLM 的 3 点有待改进的地方。

接下来，我们来看一下这些技巧会在多大程度上有效。

这里，将层结构实现为BetterRnnlm 类。

![](./img/6-36.png)


•  LSTM 层的多层化（此处为 2 层）

•　使用 Dropout（仅应用在深度方向上）

•　权重共享（Embedding 层和 Affine 层的权重共享）

In [ ]:
# ch06/better_rnnlm.py

### 前沿研究

![](./img/6-37.png)

## 小结